In [1]:
# relative import hack
import sys
sys.path.insert(0, "..")

from covid_dc.school_data import (
    scrape_articles_data,
    scrape_notifications_data,
    parse_incidents,
    data_to_dataframe,
    fix_school_names,
    append_school_levels
)

import pandas as pd

import seaborn as sns

%pylab inline

pd.options.display.max_rows = 250
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = None

Populating the interactive namespace from numpy and matplotlib


In [ ]:
incidents = scrape_notifications_data()

In [ ]:
incidents = parse_incidents(incidents)

In [ ]:
all_data = data_to_dataframe(incidents)

In [ ]:
all_data = fix_school_names(all_data)

In [ ]:
all_data = append_school_levels(all_data)

In [ ]:
metadata = pd.read_csv("../data/input/DC_schools_metadata.csv")

In [ ]:
all_data = all_data.merge(metadata, on="school", how="left")

In [ ]:
# just copypastaing this to gsheets for now
all_data[all_data.incident_date >= "2021-08-20"][
    [
        "incident_date",
        "letter_date",
        "school",
        "cases_count",
        "school_level",
        "ward",
        "incident_text",
    ]
].to_clipboard(index=False)


In [ ]:
# Dropping sample data in a folder in the repo
all_data[all_data.incident_date >= "2021-08-20"].to_csv(
    "../data/output/school_covid_data_2021_2021_school_year.csv", index=False
)

## Below here is workspace

In [ ]:
metadata.head().T